In [ ]:
import re

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import (
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


c:\projects\ods_nlp_course\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
data = pd.read_csv("data/train.csv.csv")
texts = data["text"].tolist()
rates = data["rate"].tolist()

In [ ]:
# Функция предобработки текста
def preprocess_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем лишние символы и цифры
    text = re.sub(r"[^а-яА-ЯёЁa-zA-Z\s]", "", text)
    # Удаляем лишние пробелы
    text = re.sub(r"\s+", " ", text).strip()
    return text


# Применим предобработку к данным
data["text"] = data["text"].apply(preprocess_text)


In [ ]:
# Разделение данных
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"].tolist(),
    data["rate"].tolist(),
    stratify=data["rate"],
    test_size=0.2,
    random_state=42,
)

# Загрузка токенайзера для модели
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")

# Токенизация текстов
train_encodings = tokenizer(
    train_texts, truncation=True, padding=True, max_length=128
)  # 514
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Преобразуем метки в тензоры
train_labels = torch.tensor([t - 1 for t in train_labels])
test_labels = torch.tensor([t - 1 for t in test_labels])

In [ ]:
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


# Создание датасетов
train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [ ]:
# Определение количества классов
num_labels = len(set(data["rate"]))
print(num_labels)

# Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(
    "intfloat/multilingual-e5-large-instruct", num_labels=num_labels
)

# чтобы заморозить веса модели и обучать только линейный слой после
# for param in model.base_model.parameters():
#     param.requires_grad = False


5


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large-instruct and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Функция для расчета F1 macro
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    return {"f1_macro": f1}


In [ ]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",  # Папка для сохранения модели
    eval_strategy="epoch",  # Оценка на тестовой выборке каждый эпоху
    save_strategy="epoch",
    logging_dir="./logs",  # Папка для логов
    per_device_train_batch_size=32,  # Размер batch для обучения
    per_device_eval_batch_size=32,  # Размер batch для оценки
    num_train_epochs=3,  # Количество эпох
    weight_decay=0.01,  # Коэффициент L2 регуляризации
    load_best_model_at_end=True,  # Загрузка лучшей модели по итогам оценки
    metric_for_best_model="f1_macro",  # Метрика для выбора лучшей модели
    logging_steps=10,
    # fp16=True
    # use_cpu=True
)

# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Обучение модели
trainer.train()

In [ ]:
# Оценка
from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

f1_macro = f1_score(test_labels, preds, average="macro")
f1_micro = f1_score(test_labels, preds, average="micro")
print("Test F1 Macro Score:", f1_macro)
print("Test F1 Micro Score:", f1_micro)
print("Classification Report:\n", classification_report(test_labels, preds))


100%|██████████| 305/305 [01:52<00:00,  2.71it/s]

Test F1 Macro Score: 0.5179128139688565
Test F1 Micro Score: 0.6937223877530052
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.65      0.64       828
           1       0.33      0.09      0.14       482
           2       0.44      0.53      0.48      1225
           3       0.52      0.44      0.48      1984
           4       0.83      0.89      0.86      5214

    accuracy                           0.69      9733
   macro avg       0.55      0.52      0.52      9733
weighted avg       0.68      0.69      0.68      9733



In [ ]:
ss = pd.read_csv("data/test.csv.csv")
ss["text"] = ss["text"].apply(preprocess_text)
ss.head()


,index,text
0,0,очень хороший магазин и сотрудники приятный
1,1,самый обычный продуктовый магазин есть сыры ко...
2,2,всё хорошо и комфортно
3,3,маленький филиальчик все необходимое есть две ...
4,4,плохо относятся к клиентам


In [ ]:
# Путь к папке с чекпоинтом
checkpoint_path = (
    "./results/checkpoint-3651"  # Замените на путь, где была сохранена модель
)

# Загрузка модели и токенизатора, если их нет в памяти сейчас
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path).to(device)
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")

In [ ]:
def get_embeddings_batchwise(texts, batch_size=16):
    predicted_classes = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
        batch_texts = texts[i : i + batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(device)

        # Отключаем градиенты, чтобы уменьшить использование памяти
        with torch.no_grad():
            outputs = model(**inputs)

        predicted_class = outputs.logits.argmax(-1).cpu().numpy()
        predicted_classes.append(predicted_class)

    # Объединяем все батчи в один массив
    return np.concatenate(predicted_classes)


# Получаем эмбеддинги с использованием GPU по батчам, если вдруг все сразу не влезает в память
# В этой задаче вроде не обязательно
predicted_classes = get_embeddings_batchwise(ss["text"].tolist(), batch_size=32)

In [ ]:
ss["rate"] = predicted_classes + 1
ss.head(10)

,index,text,rate
0,0,очень хороший магазин и сотрудники приятный,5
1,1,самый обычный продуктовый магазин есть сыры ко...,5
2,2,всё хорошо и комфортно,5
3,3,маленький филиальчик все необходимое есть две ...,5
4,4,плохо относятся к клиентам,1
5,5,после недавнего ремонта стало намного лучше чи...,4
6,6,мух полно особенно где мясо жуть,1
7,7,хороший продавцы,5
8,8,удобный и чистый магазинприветливый директор,5
9,9,хороший магазин хороший ассортимент чисто перс...,5


In [ ]:
ss[["index", "rate"]].to_csv("sample_submission_v2.csv", index=False)